In [3]:
import time, sys, os
from ros import rosbag
import roslib
roslib.load_manifest('sensor_msgs')
from sensor_msgs.msg import Image

import ImageFile

In [4]:
def GetFilesFromDir(dir):
    '''Generates a list of files from the directory'''
    print( "Searching directory %s" % dir )
    all = []
    left_files = []
    right_files = []
    if os.path.exists(dir):
        for path, names, files in os.walk(dir):
            for f in files:
                if os.path.splitext(f)[1] in ['.bmp', '.png', '.jpg']:
                    if 'left' in f or 'left' in path:
                        left_files.append( os.path.join( path, f ) )
                    elif 'right' in f or 'right' in path:
                        right_files.append( os.path.join( path, f ) )
                    all.append( os.path.join( path, f ) )
    return all, left_files, right_files

In [5]:
def CreateMonoBag(imgs, topic, bagname):
    '''Creates a bag file with camera images'''
    bag = rosbag.Bag(bagname, 'w')

    try:
        for i in range(len(imgs)):
            print("Adding %s" % imgs[i])
            fp = open( imgs[i], "r" )
            p = ImageFile.Parser()

            while 1:
                s = fp.read(1024)
                if not s:
                    break
                p.feed(s)

            im = p.close()
            
            #TODO: change this to read time from filename string
            Stamp = roslib.rostime.Time.from_sec(time.time())

            Img = Image()
            Img.header.stamp = Stamp
            Img.width = im_left.size[0]
            Img.height = im_left.size[1]
            Img.encoding = "rgb8"
            Img.header.frame_id = "camera"
            Img_data = [pix for pixdata in im.getdata() for pix in pixdata]
            Img_.data = Img_data

            bag.write(topic, Img, Stamp)
    finally:
        bag.close()

In [ ]:
def CreateBag(imagedir, topic, bagname):
    '''Creates the actual bag file by successively adding images'''
    all_imgs, left_imgs, right_imgs = GetFilesFromDir(imagedir)
    if len(all_imgs) <= 0:
        print("No images found in %s" % args[0])
        exit()

    CreateMonoBag(all_imgs, topic, bagname)

In [7]:
bagfiles = ['test.bag']
path = '/tmp/bags/'
imagedir = '/tmp/images/'
topic_name='/camera/image_raw'
# for bagfile in bagfiles:
#     CreateBag(imagedir, topic_name, path+bagfile)

In [15]:
for bagfile in bagfiles:
    !rosbag reindex {bagfile}

rosbag reindex test.bag
